In [1]:
print("Best accuracy for now on cross val (10-split) : 0.8417353308364544 %")

Best accuracy for now on cross val (10-split) : 0.8417353308364544 %


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Read and clean the data

In [3]:
data_train = pd.read_csv('data/train.csv')
data_score = pd.read_csv('data/test.csv')
print(data_train.shape)
print(data_score.shape)

(891, 12)
(418, 11)


In [4]:
data = data_train.append(data_score, ignore_index=True)
data.head()

/Users/antoine/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [5]:
data['Title'] = data['Name'].apply(lambda x: (x.split('.')[0]).split()[-1])

In [6]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "Countess":   "Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

data.Title = data.Title.map(normalized_titles)

In [7]:
group_age = data.groupby(['Sex','Pclass','Title'])

data.Age = group_age.Age.apply(lambda x : x.fillna(x.median()))

In [8]:
#Feel Cabin not specified with "U" for Unknow
data.Cabin = data.Cabin.fillna('U')
data.Embarked = data.Embarked.fillna(data.Embarked.mode())
data.Fare = data.Fare.fillna(data.Fare.median())

In [9]:
most_embarked = data.Embarked.value_counts().index[0]
data.Embarked = data.Embarked.fillna(most_embarked)

In [10]:
data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title
0,22.0,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Mrs
2,26.0,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Miss
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Mrs
4,35.0,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Mr


In [11]:
data.isna().sum()

Age              0
Cabin            0
Embarked         0
Fare             0
Name             0
Parch            0
PassengerId      0
Pclass           0
Sex              0
SibSp            0
Survived       418
Ticket           0
Title            0
dtype: int64

# Feature Engineering

In [12]:
#Size of family
data['FamilySize'] = data.Parch + data.SibSp + 1

In [13]:
#Keep only the 1st letters of the Cabin
data.Cabin = data.Cabin.apply(lambda x : x[0])

In [14]:
#Sex : 0 for male 1 for female
data.Sex = data.Sex.map({'male': 0, 'female': 1})

In [15]:
#Convert qualitative to quantitative
pclass_dummies = pd.get_dummies(data['Pclass'],prefix='Pclass')
title_dummies = pd.get_dummies(data['Title'],prefix='Title')
cabin_dummies = pd.get_dummies(data['Cabin'],prefix='Cabin')
embarked_dummies = pd.get_dummies(data['Embarked'],prefix='Embarked')

In [16]:
data = pd.concat([data,pclass_dummies,title_dummies,cabin_dummies,embarked_dummies],axis=1)
data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,...,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S
0,22.0,U,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,0,1,...,0,0,0,0,0,0,1,0,0,1
1,38.0,C,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,1,1,...,1,0,0,0,0,0,0,1,0,0
2,26.0,U,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,1,0,...,0,0,0,0,0,0,1,0,0,1
3,35.0,C,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,1,1,...,1,0,0,0,0,0,0,0,0,1
4,35.0,U,S,8.0500,"Allen, Mr. William Henry",0,5,3,0,0,...,0,0,0,0,0,0,1,0,0,1


In [17]:
data = data.drop(['Name','Pclass','Title','Cabin','Embarked','Ticket'],axis=1)

In [18]:
data.head()

,Age,Fare,Parch,PassengerId,Sex,SibSp,Survived,FamilySize,Pclass_1,Pclass_2,...,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S
0,22.0,7.2500,0,1,0,1,0.0,2,0,0,...,0,0,0,0,0,0,1,0,0,1
1,38.0,71.2833,0,2,1,1,1.0,2,1,0,...,1,0,0,0,0,0,0,1,0,0
2,26.0,7.9250,0,3,1,0,1.0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
3,35.0,53.1000,0,4,1,1,1.0,2,1,0,...,1,0,0,0,0,0,0,0,0,1
4,35.0,8.0500,0,5,0,0,0.0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [19]:
data_train = data.iloc[:len(data_train) , : ]
data_score = data.iloc[len(data_train): , :]

In [20]:
print(data_train.shape)
print(data_score.shape)

(891, 29)
(418, 29)


In [26]:
X = data_train.drop(['Survived'],axis=1)

In [27]:
y = data_train.Survived

In [30]:
X_score = data_score.drop(['Survived'],axis=1)

In [31]:
X_score.head()

,Age,Fare,Parch,PassengerId,Sex,SibSp,FamilySize,Pclass_1,Pclass_2,Pclass_3,...,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S
891,34.5,7.8292,0,892,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,1,0
892,47.0,7.0000,0,893,1,1,2,0,0,1,...,0,0,0,0,0,0,1,0,0,1
893,62.0,9.6875,0,894,0,0,1,0,1,0,...,0,0,0,0,0,0,1,0,1,0
894,27.0,8.6625,0,895,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,1
895,22.0,12.2875,1,896,1,1,3,0,0,1,...,0,0,0,0,0,0,1,0,0,1


# Modeling

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import zero_one_loss
from sklearn.model_selection import KFold

## Random Forest

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
n_three = [5,10,50]
depth = [2,5,8,10,12,15]
min_samples_split = [n for n in range(4, 11)]
min_samples_leaf = [n for n in range(2, 5)]
n_max = 1
d_max = 1
m_max = 1
m2_max = 1
best_score = 0.0
for n in n_three:
    for d in depth:
        for m in min_samples_split:
            for m2 in min_samples_leaf:
                len(data)
                index = np.arange(len(X))
                kf = KFold(n_splits=10)
                mean_score = 0.0
                for train, test in kf.split(index):
                    X_train, X_test, y_train, y_test = X.iloc[train,:], X.iloc[test,:], y.iloc[train], y.iloc[test]
                    random_forest_model = RandomForestClassifier(n_estimators=n, max_depth=d,min_samples_split=m,min_samples_leaf=m2)
                    random_forest_model.fit(X_train, y_train)
                    y_res = random_forest_model.predict(X_test)
                    rate = zero_one_loss(y_res,y_test)
                    mean_score += (1-rate)
                mean_score = mean_score/10
                print("For",n,'three of depth',d,':')
                print("Accuracy : ",mean_score,'%')
                if (mean_score > best_score):
                    n_max = n
                    d_max = d
                    m_max = m
                    m2_max = m2
                    best_score = mean_score

For 5 three of depth 2 :
Accuracy :  0.7811735330836455 %
For 5 three of depth 2 :
Accuracy :  0.7733333333333333 %
For 5 three of depth 2 :
Accuracy :  0.7788764044943821 %
For 5 three of depth 2 :
Accuracy :  0.8035830212234707 %
For 5 three of depth 2 :
Accuracy :  0.7934831460674157 %
For 5 three of depth 2 :
Accuracy :  0.7912234706616729 %
For 5 three of depth 2 :
Accuracy :  0.788976279650437 %
For 5 three of depth 2 :
Accuracy :  0.7732958801498128 %
For 5 three of depth 2 :
Accuracy :  0.7812109862671661 %
For 5 three of depth 2 :
Accuracy :  0.7788639200998752 %
For 5 three of depth 2 :
Accuracy :  0.7990886392009988 %
For 5 three of depth 2 :
Accuracy :  0.8103245942571785 %
For 5 three of depth 2 :
Accuracy :  0.7733333333333334 %
For 5 three of depth 2 :
Accuracy :  0.7722347066167292 %
For 5 three of depth 2 :
Accuracy :  0.781123595505618 %
For 5 three of depth 2 :
Accuracy :  0.7891136079900125 %
For 5 three of depth 2 :
Accuracy :  0.7834831460674158 %
For 5 three of d

For 10 three of depth 2 :
Accuracy :  0.7935205992509363 %
For 10 three of depth 2 :
Accuracy :  0.7901747815230962 %
For 10 three of depth 2 :
Accuracy :  0.7979525593008738 %
For 10 three of depth 2 :
Accuracy :  0.78458177278402 %
For 10 three of depth 2 :
Accuracy :  0.787865168539326 %
For 10 three of depth 2 :
Accuracy :  0.7946067415730338 %
For 10 three of depth 5 :
Accuracy :  0.830561797752809 %
For 10 three of depth 5 :
Accuracy :  0.8350312109862671 %
For 10 three of depth 5 :
Accuracy :  0.8271785268414481 %
For 10 three of depth 5 :
Accuracy :  0.8271785268414483 %
For 10 three of depth 5 :
Accuracy :  0.8327840199750313 %
For 10 three of depth 5 :
Accuracy :  0.8204619225967541 %
For 10 three of depth 5 :
Accuracy :  0.8182521847690387 %
For 10 three of depth 5 :
Accuracy :  0.8260549313358302 %
For 10 three of depth 5 :
Accuracy :  0.8260424469413234 %
For 10 three of depth 5 :
Accuracy :  0.8204244694132334 %
For 10 three of depth 5 :
Accuracy :  0.8305243445692883 %
F

For 50 three of depth 5 :
Accuracy :  0.836154806491885 %
For 50 three of depth 5 :
Accuracy :  0.8316604244694134 %
For 50 three of depth 5 :
Accuracy :  0.832784019975031 %
For 50 three of depth 5 :
Accuracy :  0.8316604244694131 %
For 50 three of depth 5 :
Accuracy :  0.8294257178526842 %
For 50 three of depth 5 :
Accuracy :  0.8282896379525594 %
For 50 three of depth 5 :
Accuracy :  0.8249063670411985 %
For 50 three of depth 5 :
Accuracy :  0.8339076154806492 %
For 50 three of depth 5 :
Accuracy :  0.8294007490636706 %
For 50 three of depth 5 :
Accuracy :  0.8271785268414481 %
For 50 three of depth 5 :
Accuracy :  0.8294132334581773 %
For 50 three of depth 5 :
Accuracy :  0.8327715355805243 %
For 50 three of depth 5 :
Accuracy :  0.8294257178526842 %
For 50 three of depth 5 :
Accuracy :  0.8260549313358302 %
For 50 three of depth 8 :
Accuracy :  0.8372908863920101 %
For 50 three of depth 8 :
Accuracy :  0.830561797752809 %
For 50 three of depth 8 :
Accuracy :  0.8327965043695382 %


In [36]:
print("Best accurate is given for ",n_max,"three of depth",d_max,'min samples split',m,' and min samples leaf',m2,"with an accurency of",best_score,"% in cross validation")

Best accurate is given for  50 three of depth 12 min samples split 10  and min samples leaf 4 with an accurency of 0.8440324594257179 % in cross validation


# Submission

In [37]:
random_forest_model = RandomForestClassifier(n_estimators=50, max_depth=12,min_samples_split=10,min_samples_leaf=4)
random_forest_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=12, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [38]:
predictions = random_forest_model.predict(X_score).astype(int)

In [39]:
output = pd.DataFrame({'PassengerId': data_score.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [40]:
output

,PassengerId,Survived
891,892,0
892,893,0
893,894,0
894,895,0
895,896,1
896,897,0
897,898,1
898,899,0
899,900,1
900,901,0


In [1]:
data.name

NameError: name 'data' is not defined